In [ ]:
import os
import traceback
import time
import sys
import json
from random import randint
from urllib.request import urlretrieve
import flickrapi
# API settings
flickr_api_key = 'xxx' #change 'xxx' with your flickr_api_key
secret_key = 'xxx' #change 'xxx' with your secret_key

def download_flickr_photos(keywords, size='medium', max_nb_img=-1):
    
    """
    Downloads images based on keyword search on the Flickr website
    
    Parameters
    ----------
    keywords : string, list of strings
        Keyword to search for or a list of keywords should be given.
    size : one of the following strings 'thumbnail', 'square', 'medium', default: 'original'.
        Size of the image to download. In this function we only provide
        four options. More options are explained at 
        http://librdf.org/flickcurl/api/flickcurl-searching-search-extras.html
    max_nb_img : int, default: -1
        Maximum number of images per keyword to download. If given a value of -1, all images
        will be downloaded
    
    Returns
    ------
    Images found based on the keyword are saved in a separate subfolder.
    
    Notes
    -----
    This function uses the Python package flickrapi and its walk method. 
    FlickrAPI.walk has same parameters as FlickrAPI.search
    http://www.flickr.com/services/api/flickr.photos.search.html
    
    To use the Flickr API a set of API keys needs to be created on 
    https://www.flickr.com/services/api/misc.api_keys.html
    """
    if not (isinstance(keywords, str) or isinstance(keywords, list)):
        raise AttributeError('keywords must be a string or a list of strings')
        
    if not (size in ['thumbnail', 'square', 'medium', 'original']):
        raise AttributeError('size must be "thumbnail", "square", "medium" or "original"')
                             
    if not (max_nb_img == -1 or (max_nb_img > 0 and isinstance(max_nb_img, int))):
        raise AttributeError('max_nb_img must be an integer greater than zero or equal to -1')
    
    flicker = flickrapi.FlickrAPI (flickr_api_key, secret_key, format = 'parsed-json')
    
    if isinstance(keywords, str):
        keywords_list = []
        keywords_list.append(keywords)
    else:
        keywords_list = keywords
        
    if size == 'thumbnail':
        size_url = 'url_t'
    elif size == 'square':
        size_url = 'url_q'
    elif size == 'medium':
        size_url = 'url_c'
    elif size == 'original':
        size_url = 'url_o'
    
    for keyword in keywords_list: 
        count = 0
        if __name__ == '__main__':   
            print ('accessing ...')
            response = flicker.photos.search (
                text = keyword,
                media = 'photos',
                sort = 'relevance',
                license = '1,2,4,5',
                per_page = 500,
                extras = 'url_m, date_taken, license, owner_name, geo')
            photos = response ['photos']
            try:
                out_dirpath = os.path.join ('image_data', keyword)
                os.makedirs (out_dirpath, exist_ok = True)               
                for photo in photos ['photo']:
                    t = randint(1, 3)
                    time.sleep(t)
                    count += 1
                    if max_nb_img != -1:
                        if count > max_nb_img:
                            print('Reached maximum number of images to download')
                            break
                    url = photo ['url_m']
                    out_path = os.path.join (out_dirpath, str(count) + '_'+ photo ['id'] + '.jpg')
                    print ('getting image ...'+ str(count)  + '_' + url)
                    urlretrieve (url, out_path)
            except Exception as e:
                traceback.print_exc ()
            with open ('photo _ {}. json'.format (keyword),'w') as f:
                json.dump (photos, f, sort_keys = True, indent = 4)
            print("Total images downloaded:", str(count) + '_' + keyword)
        
flowers = ['Cypripedium guttatum','Corallorhiza mertensiana','Corallorhiza wisteriana','Encyclia tampensis','Oeceoclades maculata','Triphora craigheadii']
for flower in flowers:
    download_flickr_photos(flower)